# Perfiles de comportamiento y minado de patrones

In [1]:
import numpy as np
import datetime
from datetime import date
import json
import pylab
import pandas as pd
import matplotlib.pyplot as plt
import os, sys, logging, warnings, time

In [2]:
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), "behavior_patterns/src/"))
import profile_builder as profile_manager

## Contruccion de "Footprints"

En esta etapa contruiremos los "footprints" basicos (tensores dimesionales) de cada cliente a partir de sus datos transaccionales, la dimesiones del footprint son elegidas por el usuario, nosotros tenemos los siguientes casos de estudio:

**Datos de bancarios**:
 - Sesion de TX
   - s = {cliente, timestamp, Monto}  
 - Cada cliente tiene una secuencia de sesiones de TX (S)
    - S = { s1, s2 , s3, ..., sn}                n: # de sesiones de cada cliente

**Datos mobiles**:
 - Sesion de TX
   - s = {cliente, timestamp, Monto}  
 - Cada cliente tiene una secuencia de sesiones de TX (S)
    - S = { s1, s2 , s3, ..., sn}    

In [3]:
to_read = 'bank_trx' 

In [4]:
sessions = profile_manager.session_data(font=to_read)

categories = list(np.unique(sessions['mccg']))
turns = list(np.unique(sessions['turn']))
day_of_week = list(np.unique(sessions['weekday']))

sessions.head(3)

,client_id,date,año,mes,dia,hora,merchant_departement,merchant_province,merchant_district,mccg,mccg_name,mcc,quantity,amount_usd,amount_sol,week,weekday,day,turn
0,+++g8j9k+5A=,2016-07-07 01:08:05,2016,7,7,1,LIMA,LIMA,San Juan De Luriganc,9,Clothing Stores,5655,1,13.875570,45.0,27,3,Thursday,1
1,++/oQ9Lb9dI=,2017-05-29 21:05:40,2017,5,29,21,LIMA,LIMA,San Juan De Luriganc,7,Utility Services,4812,1,32.067984,104.0,22,0,Monday,4
2,++1XMtcwMec=,2016-09-24 17:45:25,2016,9,24,17,LIMA,LIMA,San Juan De Luriganc,8,Retail Outlet Services,5533,1,55.502280,180.0,38,5,Saturday,3


In [5]:
#sessions

In [ ]:
footprints = profile_manager.footprints(font=to_read,count_session=False)

Procesando archivo
Creando archivo nuevo
User quantity: 175118
Procesando en parallelo ...


In [ ]:
footprints.head()
#footprints

In [ ]:
# footprints[['footprint_id','year','week','profile_id','category','turn','size','c10t4d0']]

In [ ]:
footprints[(footprints['size']==0)]

In [ ]:
sessions[sessions['client_id']=='+TNzsXNd57o=']

In [ ]:
footprints[footprints['footprint_id']=='++1XMtcwMec=']

In [ ]:
footprints[footprints['footprint_id']=='+TNzsXNd57o=']

In [ ]:
pd.get_dummies(footprints)

In [ ]:
for col in footprints.colums:
    print(col)